In [ ]:
from google.colab import files
uploaded = files.upload()

Saving train_test.csv to train_test.csv


In [ ]:
!pip install nlppreprocess
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
%matplotlib inline
import string
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from wordcloud import WordCloud
import string
from nlppreprocess import NLP
import pickle
# splitting data
from sklearn.model_selection import train_test_split
# Features Extraction
from sklearn.feature_extraction.text import CountVectorizer
# Features Extraction
from sklearn.feature_extraction.text import TfidfVectorizer

# Evaluation
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

# models
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
# from sklearn.svm import SVC

from sklearn import preprocessing

from sklearn.svm import SVC

# # Features Extraction word2vec
# from gensim.models import Word2Vec

# # Dimension reduction
# from sklearn.decomposition import PCA

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
import io
df_file = pd.read_csv(io.BytesIO(uploaded['train_test.csv']))
df_file.dropna(inplace=True)
print(len(df_file))
df_file.head()

31014


,textID,text,sentiment
0,cb774db0d1,"I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,negative
2,088c60f138,my boss is bullying me...,negative
3,9642c003ef,what interview! leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...",negative


In [ ]:
df_file = df_file.sample(frac=1).reset_index(drop=True) # shuffel
df_file = df_file[:25000]
print(len(df_file))
df_file.head()

25000


,textID,text,sentiment
0,a9f8ba080b,it was t4 i think hannah ****,neutral
1,082abf7f5a,Aw .. the grillz in my background look like pl...,neutral
2,89613d7a9d,I just spent 45 minutes at the book store. I o...,neutral
3,5ffb19ae8f,Gnight shar <(` `<)Vega(>` `)>,neutral
4,5792dd85ca,I may be telling myself that cos he never rep...,negative


# Cleaning

In [ ]:


def one_string_lower_conversation(sentence):
    '''
    Argument:
        text as string of words
    return:
        lower of this string
    '''
    return sentence.lower()
  
def all_string_lower_conversation(text_list):
    '''
    Argument:
        list of strings and each of these strings does contain some of words
    return:
        lower each string in this list
    '''
    text_list = [one_string_lower_conversation(sentence) for sentence in text_list]
    return text_list


def one_string_remove_punctuation(sentence):
    '''
    Argument:
        string of words
    reutrn:
        string without punctuation like [.!?] and others
    '''
    sentence = re.sub("""(@\w+[1-9:'\/"]? |http:\/\/\w+.\w+.\w+\/)""", '', sentence)
    sentence = sentence.split(' ')
    
    strs = ''
    punctuations = string.punctuation
    for word in sentence:
        word = re.sub('[^\w\s+]',' ',word)
        strs += word + ' '
    translator = str.maketrans('', '', punctuations)
    strs.translate(translator)
    return strs

def all_strings_remove_punctuation(text_list):
    '''
    Argument:
        list of strings 
    reutrn:
        list of strings without punctuation like [.!?] and others
    '''
    text_list = [one_string_remove_punctuation(sentence) for sentence in text_list]
    return text_list


def one_string_tokenization(sentence):
    '''
    Argument:
        String of words
    return:
        list of words
    '''
    sentence = word_tokenize(sentence)
    return sentence


def all_string_tokenization(text_list):
    '''
    Argument:
        list of Strings
    return:
        list of strings and every string is list of words
    '''
    text_list = [one_string_tokenization(sentence) for sentence in text_list]
    return text_list


def one_string_Lemmatizing(sentence):
    '''
    Argument:
        String of words
    return:
        list of words with Lemmatizing
    '''
    sentence = one_string_tokenization(sentence)
    lemmatizer = WordNetLemmatizer()
    sentence = [lemmatizer.lemmatize(word) for word in sentence]
    return sentence

def all_string_Lemmatizing(text_list):
    '''
    Argument:
        list of strings
    return:
        list of strings with steming which the root of the word in each string
    '''
    text_list = [one_string_Lemmatizing(sentence) for sentence in text_list]
    return text_list


def one_string_stop_words(sentence):
    '''
    Argument:
        string of words
    return:
        remove stop words from this string like this, did
        but other words like not, no dont remove
	'''
    stop_words = NLP().stopword_list
    sentence = sentence.split(' ')
    updated_sentence = ''
    for word in sentence:
    	if word not in stop_words:
    		updated_sentence += word + ' '
    return updated_sentence






def all_string_stop_words(text_list):
  '''
  Argument:
      list of string
  return:
      list of string without stop words
  '''
      
  text_list = [one_string_stop_words(sentence) for sentence in text_list]
  return text_list

def one_string_un_tokenization(sentence):
    '''
    Argument:
        list of words
    return:
        string of words
    '''
    sentence = " ".join(sentence)
    return sentence
    
def all_string_un_tokenization(text_list):
    '''
    Argument:
        list of words
    return:
        string of words
    '''
    text_list = [one_string_un_tokenization(sentence) for sentence in text_list]
    return text_list

def english_pip_line(text_list):
    text_list = all_string_lower_conversation(text_list)
    text_list = all_strings_remove_punctuation(text_list)
    text_list = all_string_stop_words(text_list)
    return text_list



In [ ]:
print(np.unique(df_file['sentiment']))
print(df_file.columns)
text_list = list(df_file['text'])
# text_list = english_pip_line(text_list)
len(text_list)


['negative' 'neutral' 'positive']
Index(['textID', 'text', 'sentiment'], dtype='object')


25000

In [ ]:
X = text_list
y = df_file['sentiment']

In [ ]:
def tfidf_vectorizer(df):
    '''
    Argumen:
        df dataframe of multiple reviews
    return:
        Train & test arrays that can fir to the model
    '''
# I fit the vector to all of the data
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_vectorizer = tfidf_vectorizer.fit(df) 
    word_idf_weights = tfidf_vectorizer.idf_
    print("Our 10 words weights\n\n",word_idf_weights[:10])
    return tfidf_vectorizer

In [ ]:
tfidf_vectorizer = tfidf_vectorizer(text_list) 
pickle.dump(tfidf_vectorizer, open("tfidf_vectorizer_25000.pickle", "wb"))
files.download("tfidf_vectorizer_25000.pickle")
tfidf_vectorizer = pickle.load(open("tfidf_vectorizer_25000.pickle", "rb"))

Our 10 words weights

 [ 8.18223212  8.23629935 10.43352392  9.74037674 10.43352392 10.43352392
  9.74037674  9.51723319  8.92944653 10.43352392]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# convert y from text to label encoder and we can see all entiment labels now
LabelEncoder = preprocessing.LabelEncoder()
LabelEncoder.fit(y)
list(LabelEncoder.classes_)
y = LabelEncoder.transform(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)
print("Our training data now are: " + str(len(X_train))  + " Reviews")
print("Our testing data now are: " + str(len(X_test))  + " Reviews")
print("Our training data now are: " + str(len(y_train))  + " labels")
print("Our testing data now are: " + str(len(y_test))  + " labels")

Our training data now are: 20000 Reviews
Our testing data now are: 5000 Reviews
Our training data now are: 20000 labels
Our testing data now are: 5000 labels


In [ ]:
training_data = tfidf_vectorizer.transform(X_train)
training_data  = training_data.toarray()
testing_data = tfidf_vectorizer.transform(X_test)
testing_data  = testing_data.toarray()

In [ ]:
clf_MultinomialNB = MultinomialNB()
model = clf_MultinomialNB.fit(training_data, y_train)
predict = model.predict(training_data)
print("F1 score of our training data is: ", f1_score(y_train, predict, average='micro'))
predict = model.predict(testing_data)
print("F1 score of our testing data is: ", f1_score(y_test, predict, average='micro'))

F1 score of our training data is:  0.7656
F1 score of our testing data is:  0.602


In [ ]:
pickle.dump(model, open("MultinomialNB_61%_25000.pickle", "wb"))
files.download('MultinomialNB_61%_25000.pickle')
count_vectorizer = pickle.load(open("MultinomialNB_61%_25000.pickle", "rb"))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
clf_LogisticRegression = LogisticRegression(penalty='l2', solver='sag')
logistic_model = clf_LogisticRegression.fit(training_data, y_train)
predict = logistic_model.predict(training_data)
print("F1 score of our training data is: ", f1_score(y_train, predict, average='micro'))
predict = logistic_model.predict(testing_data)
print("F1 score of our testing data is: ", f1_score(y_test, predict, average='micro'))

F1 score of our training data is:  0.82325
F1 score of our testing data is:  0.6822


In [ ]:
pickle.dump(model, open("logistic_model_sag_68%_25000.pickle", "wb"))
# files.download('logistic_model_sag_68%_25000.pickle')
count_vectorizer = pickle.load(open("logistic_model_sag_68%_25000.pickle", "rb"))

In [ ]:
clf_LogisticRegression = LogisticRegression(penalty='l2', solver='liblinear')
logistic_model = clf_LogisticRegression.fit(training_data, y_train)
predict = logistic_model.predict(training_data)
print("F1 score of our training data is: ", f1_score(y_train, predict, average='micro'))
predict = logistic_model.predict(testing_data)
print("F1 score of our testing data is: ", f1_score(y_test, predict, average='micro'))

F1 score of our training data is:  0.78825
F1 score of our testing data is:  0.6846


In [ ]:
pickle.dump(model, open("logistic_model_liblinear_68%_25000.pickle", "wb"))
# files.download('logistic_model_liblinear_68%_25000.pickle')
count_vectorizer = pickle.load(open("logistic_model_liblinear_68%_25000.pickle", "rb"))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>